### Import Packages

In [34]:
import os
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as functional
from sklearn.model_selection import train_test_split

### Load Data

In [35]:
def load_data(base_dirs, stack):
    all_data = []
    all_labels = []
    
    # 遍歷每個目錄
    for label, directory in enumerate(base_dirs):
        dir_path = Path(directory)
        print(f"Processing directory: {dir_path} (Label: {label})")
        
        # 取得目錄下所有的 .npy 檔案
        npy_files = list(dir_path.glob("*.npy"))
        all_data = []
        all_labels = []
        for npy_file in npy_files:
            try:
                # 載入 npy 檔案
                data = np.load(npy_file)
                
                # 檢查數據是否是 2D
                if len(data.shape) != 2:
                    raise ValueError(f"Expected 2D data, got shape {data.shape}")
                if data.shape[1] != 157:
                    data = data[:, 0:157]
                # 將數據和標籤加入列表
                all_data.append(data)
                all_labels.append(label)  # 每個檔案對應一個標籤
                
                # print(f"Loaded {npy_file.name}: Shape {data.shape}")
                
            except Exception as e:
                print(f"Error loading {npy_file}: {str(e)}")

        
        all_data = np.stack(all_data, axis=0)  # (N, H, W)
        all_labels = np.array(all_labels)  # (N,)
    
        all_data = np.expand_dims(all_data, axis=1)
    # 正規化數據到 [0, 1] 區間
    # all_data = all_data.astype(np.float32) / 255.0
    print(f"\nFinal dataset shape: {all_data.shape}")
    print(f"Labels shape: {all_labels.shape}")
    print(f"Unique labels: {np.unique(all_labels)}")
    
    return all_data, all_labels

In [36]:
class MelDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.FloatTensor(data)
        self.labels = torch.LongTensor(labels)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

### Model

In [37]:
class MelClassifier(nn.Module):
    def __init__(self, input_channels):
        super(MelClassifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        self.classifier = None  # 延遲初始化
        
    def forward(self, x):
        x = self.features(x)
        if self.classifier is None:  # 第一次執行 forward 時初始化
            num_features = x.view(x.size(0), -1).size(1)
            self.classifier = nn.Sequential(
                nn.Flatten(),
                nn.Linear(num_features, 512),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(512, 3)
            ).to(x.device)
        x = self.classifier(x)
        return x

In [38]:
# Hyperparameters
learning_rate = 0.005
num_epochs = 20
batch_size = 64

In [39]:
class MelDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.FloatTensor(data)
        self.labels = torch.LongTensor(labels)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

### Training

In [40]:
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        # print(f"Model output shape: {outputs.shape}")
        # print(f"Labels shape: {labels.shape}")
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(train_loader), 100. * correct / total



def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(test_loader), 100. * correct / total
    

In [41]:
def save_classified_results_npy(model, test_data, output_dirs, device):
    """
    保存分類結果為 .npy 檔案到不同資料夾
    """
    model.eval()
    os.makedirs(output_dirs[0], exist_ok=True)  # human_output
    os.makedirs(output_dirs[1], exist_ok=True)  # machine_output
    os.makedirs(output_dirs[2], exist_ok=True)  # nature_output
    
    for idx, sample in enumerate(test_data):
            # 增加 batch 和通道維度 (1, C, H, W)
            # sample = np.expand_dims(sample, axis=0)  # (1, H, W)
            sample = np.expand_dims(sample, axis=1)  # (1, 1, H, W)
            print(sample.shape)
            sample = torch.FloatTensor(sample).to(device)
            
            # 推理
            output = model(sample)
            predicted = output.argmax(dim=1).item()
            
            # 保存結果
            output_dir = output_dirs[predicted]
            Path(output_dir).mkdir(parents=True, exist_ok=True)
            np.save(f"{output_dir}/sample_{idx}.npy", sample.cpu().numpy())

In [42]:
# label of each directory: 0, 1, 2
base_dirs = ["../human/mixed", "../machine/mixed", "../nature/mixed"]

# load data
data, labels = load_data(base_dirs, True)

# train, test split
X_train, X_test, y_train, y_test = train_test_split(
        data, labels, test_size=0.2, random_state=42, stratify=labels)
train_dataset = MelDataset(X_train, y_train)
test_dataset = MelDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)


Processing directory: ..\human\mixed (Label: 0)
Processing directory: ..\machine\mixed (Label: 1)
Processing directory: ..\nature\mixed (Label: 2)

Final dataset shape: (7500, 1, 128, 157)
Labels shape: (7500,)
Unique labels: [2]


In [43]:
# for train_image, labels in train_loader:
#     print(train_image.shape)
#     break

In [44]:

input_channel = data.shape[1]
model = MelClassifier(input_channel)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_losses = []
train_accs = []
test_losses = []
test_accs = []
for epoch in range(num_epochs):
        train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, device)
        test_loss, test_acc = evaluate_model(model, test_loader, criterion, device)
        
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        test_losses.append(test_loss)
        test_accs.append(test_acc)
        
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')

final_test_loss, final_test_acc = evaluate_model(model, test_loader, criterion, device)
print(f"\nFinal Test Accuracy: {final_test_acc:.2f}%")
print(f"Final Test Loss: {final_test_loss:.4f}")


    


Epoch [1/20]
Train Loss: 0.0290, Train Acc: 99.28%
Test Loss: 0.0000, Test Acc: 100.00%
Epoch [2/20]
Train Loss: 0.0124, Train Acc: 99.97%
Test Loss: 0.0000, Test Acc: 100.00%
Epoch [3/20]
Train Loss: 0.0000, Train Acc: 100.00%
Test Loss: 0.0000, Test Acc: 100.00%
Epoch [4/20]
Train Loss: 0.0000, Train Acc: 100.00%
Test Loss: 0.0000, Test Acc: 100.00%
Epoch [5/20]
Train Loss: 0.0000, Train Acc: 100.00%
Test Loss: 0.0000, Test Acc: 100.00%
Epoch [6/20]
Train Loss: 0.0000, Train Acc: 100.00%
Test Loss: 0.0000, Test Acc: 100.00%
Epoch [7/20]
Train Loss: 0.0000, Train Acc: 100.00%
Test Loss: 0.0000, Test Acc: 100.00%
Epoch [8/20]
Train Loss: 0.0000, Train Acc: 100.00%
Test Loss: 0.0000, Test Acc: 100.00%
Epoch [9/20]
Train Loss: 0.0000, Train Acc: 100.00%
Test Loss: 0.0000, Test Acc: 100.00%
Epoch [10/20]
Train Loss: 0.0000, Train Acc: 100.00%
Test Loss: 0.0000, Test Acc: 100.00%
Epoch [11/20]
Train Loss: 0.0014, Train Acc: 99.98%
Test Loss: 0.0000, Test Acc: 100.00%
Epoch [12/20]
Train Lo

In [45]:
# 保存測試分類結果
test_data_dir = ["../test_data"]
test_data, _ = load_data(test_data_dir, False)
output_dirs = {0: "human_output", 1: "machine_output", 2: "nature_output"}
save_classified_results_npy(model, test_data, output_dirs, device)

Processing directory: ..\test_data (Label: 0)

Final dataset shape: (105, 1, 128, 157)
Labels shape: (105,)
Unique labels: [0]
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1, 128, 157)
(1, 1,

In [46]:
torch.save(model, "classify_model.pth") # TODO